In [22]:
# import your libraries
import tweepy
import json
import requests
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import datetime as dt

In [23]:
# open file with twitter keys (only need bearer token for this example)
credentials = "keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [24]:
bearer_token = api_tokens["bearer_token"]

In [25]:
# create the client object which will allow you to interact with the Twitter API
client = tweepy.Client(bearer_token=bearer_token)

In [26]:
duolingo_data = client.search_recent_tweets(
    # building a query string: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
    query = "#Duolingo365 -is:retweet lang:en",
    max_results = 100
)

duolingo_data.data

[<Tweet id=1602267806891925504 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/javsY1UXUf'>,
 <Tweet id=1602267516834877441 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/UhlDZqJh7k'>,
 <Tweet id=1602266572474785792 text='I’m a Superstar Learner! What’s your Duolingo learner style? #Duolingo365 https://t.co/uAtWzS3tjN'>,
 <Tweet id=1602265050151981056 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/ZpD7n28K70'>,
 <Tweet id=1602263141504655372 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/e1h29c99gj'>,
 <Tweet id=1602261793124110337 text='I’m a Learning Legend! What’s your Duolingo learner style? #Duolingo365 https://t.co/3FWgowbWeN'>,
 <Tweet id=1602261274133377027 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/42rqyUqbRm'>,
 <Tweet id=1602259327066116096 text=

In [27]:
# create a list to only store the tweets, none of the metadata
tweets_duolingo = []

for tweet in duolingo_data.data:
    tweets_duolingo.append(tweet.text)
    
tweets_duolingo

['Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/javsY1UXUf',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/UhlDZqJh7k',
 'I’m a Superstar Learner! What’s your Duolingo learner style? #Duolingo365 https://t.co/uAtWzS3tjN',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/ZpD7n28K70',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/e1h29c99gj',
 'I’m a Learning Legend! What’s your Duolingo learner style? #Duolingo365 https://t.co/3FWgowbWeN',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/42rqyUqbRm',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/PXZ0eqEwfk',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/kGAYqBHas0',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://

In [28]:
morgan_data = client.search_recent_tweets(
    query="Piers Morgan -is:retweet lang:en",
    max_results = 100
)

morgan_data.data

[<Tweet id=1602270900660981763 text='@TimelineCR7 @carrusel Go cry to piers morgan'>,
 <Tweet id=1602270575291867137 text='Clare Piers Morgan Jimmy #NBA #NBA #开云体育 https://t.co/ul5Z2IRFQj'>,
 <Tweet id=1602270417577803777 text='@Sports_Predics @Messi30_19_10 @KieranCFC88 I think you forgot the piers morgan show'>,
 <Tweet id=1602270345364373505 text="@RMBlancoZz Possibly, if he hadn't gone on piers morgan to whine like a little b***h">,
 <Tweet id=1602270272731713537 text='@piersmorgan @Cristiano Extremely rare Piers Morgan W'>,
 <Tweet id=1602269771751247875 text='@UTDTrey That’s the difference. From Piers Morgan to Utdtrey, everyone obsessed with Ronaldo win &amp; Messi loss so that fanboys can stamp their claims with stats. We want Messi to win cuz he DESERVES it &amp; lad has been proving it on the pitch. Banters if he lose? Don’t care. He did his best.'>,
 <Tweet id=1602269482503835651 text="@Lord_Sugar @piersmorgan People seem to have forgotten all the crap Piers Morgan has done\

In [29]:
morgan_tweets = []
for tweet in morgan_data.data:
    morgan_tweets.append(tweet.text)
    
morgan_tweets

['@TimelineCR7 @carrusel Go cry to piers morgan',
 'Clare Piers Morgan Jimmy #NBA #NBA #开云体育 https://t.co/ul5Z2IRFQj',
 '@Sports_Predics @Messi30_19_10 @KieranCFC88 I think you forgot the piers morgan show',
 "@RMBlancoZz Possibly, if he hadn't gone on piers morgan to whine like a little b***h",
 '@piersmorgan @Cristiano Extremely rare Piers Morgan W',
 '@UTDTrey That’s the difference. From Piers Morgan to Utdtrey, everyone obsessed with Ronaldo win &amp; Messi loss so that fanboys can stamp their claims with stats. We want Messi to win cuz he DESERVES it &amp; lad has been proving it on the pitch. Banters if he lose? Don’t care. He did his best.',
 "@Lord_Sugar @piersmorgan People seem to have forgotten all the crap Piers Morgan has done\nThe soldiers, Milly Dowler, they seem to think he's some sort of hero now\nHe just jumps on the latest bandwagon",
 'Harry&amp;Megan. Our privately owned print media is the worst in the world: corrupt, venal, proven criminal. It created the condition

In [30]:
import random

# create a function to format our tweet into a collection of key-value pairs (ie features)
def setup_features(text):
    # create array of keywords
    # make text lowercase
    text = text.lower()
    # split the text into an array of words
    words_list = text.split()
    # this isn't nessecary but will make the next part shorter
    s_words = stopwords.words('english')
    keywords = [ term for term in words_list if term not in s_words and 'http' not in term] 
    random.shuffle(keywords)
    while len(keywords) < 3:
        keywords.append('None')
    return {'first_keyword': keywords[0], 'second_keyword': keywords[1], 'third_keyword': keywords[2]}

In [31]:
# create a list of tuples containing the featureset and the label
# features and label list will contain my features and my label
features_and_labels = []
for tweet in tweets_duolingo:
    features_and_labels.append(  (setup_features(tweet), 'Duolingo')    )

In [32]:
for tweet in morgan_tweets:
    features_and_labels.append(  (setup_features(tweet), 'Piers Morgan')    )

In [33]:
# print out our features and labels
features_and_labels

[({'first_keyword': 'look',
   'second_keyword': 'duolingo',
   'third_keyword': 'learned'},
  'Duolingo'),
 ({'first_keyword': 'much', 'second_keyword': '2022!', 'third_keyword': 'do?'},
  'Duolingo'),
 ({'first_keyword': 'style?',
   'second_keyword': 'duolingo',
   'third_keyword': 'superstar'},
  'Duolingo'),
 ({'first_keyword': '2022!', 'second_keyword': 'look', 'third_keyword': 'do?'},
  'Duolingo'),
 ({'first_keyword': 'look',
   'second_keyword': '#duolingo365',
   'third_keyword': '2022!'},
  'Duolingo'),
 ({'first_keyword': 'learning',
   'second_keyword': '#duolingo365',
   'third_keyword': 'i’m'},
  'Duolingo'),
 ({'first_keyword': 'much',
   'second_keyword': 'look',
   'third_keyword': '2022!'},
  'Duolingo'),
 ({'first_keyword': 'do?',
   'second_keyword': 'learned',
   'third_keyword': '2022!'},
  'Duolingo'),
 ({'first_keyword': 'look',
   'second_keyword': 'much',
   'third_keyword': '#duolingo365'},
  'Duolingo'),
 ({'first_keyword': 'do?',
   'second_keyword': 'duol

In [34]:
# ALWAYS SHUFFLE YOUR FEATURES AND LABELS!!!!
random.shuffle(features_and_labels)

In [35]:
# print out our shuffled features and labels
features_and_labels

[({'first_keyword': '2022!',
   'second_keyword': '#duolingo365',
   'third_keyword': 'duolingo'},
  'Duolingo'),
 ({'first_keyword': 'morgan',
   'second_keyword': 'hope',
   'third_keyword': 'manipulated'},
  'Piers Morgan'),
 ({'first_keyword': 'much', 'second_keyword': '2022!', 'third_keyword': 'do?'},
  'Duolingo'),
 ({'first_keyword': 'what’s',
   'second_keyword': '#duolingo365',
   'third_keyword': 'learner'},
  'Duolingo'),
 ({'first_keyword': '@piersmorgan',
   'second_keyword': 'also',
   'third_keyword': 'health,'},
  'Piers Morgan'),
 ({'first_keyword': 'even',
   'second_keyword': 'spout',
   'third_keyword': 'morgan'},
  'Piers Morgan'),
 ({'first_keyword': 'much',
   'second_keyword': 'duolingo',
   'third_keyword': '#duolingo365'},
  'Duolingo'),
 ({'first_keyword': 'journalists',
   'second_keyword': 'newspaper',
   'third_keyword': 'often'},
  'Piers Morgan'),
 ({'first_keyword': 'oppostion',
   'second_keyword': '@akidaawal',
   'third_keyword': 'interview'},
  'Pie

In [36]:
# split 80:20 training data and testing data

import math

# get how many elements in my list is 80%
split_num = math.floor(len(features_and_labels) * .8)
split_num

158

In [37]:
# 80% of my data
training_set = features_and_labels[:split_num]
# 20% of my data
testing_set = features_and_labels[split_num:]

In [38]:
import nltk
# create our classifier

# My baseline is 50% because I have two labels/outcomes - we are hoping to do better than the baseline!
my_classifier = nltk.NaiveBayesClassifier.train(training_set)

In [39]:
# grab a random tweet from Twitter and see if it is classified correctly
my_classifier.classify(setup_features("Big Piers Morgan getting told off by specsavers has made my day."))

'Piers Morgan'

In [42]:
# test our accuracy
nltk.classify.accuracy(my_classifier, testing_set)

0.9

In [43]:
my_classifier.show_most_informative_features(10)

Most Informative Features
           third_keyword = 'look'         Duolin : Piers  =      6.2 : 1.0
          second_keyword = 'i’m'          Piers  : Duolin =      1.0 : 1.0
           third_keyword = 'shame'        Piers  : Duolin =      1.0 : 1.0
